```
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%%capture
!pip install --upgrade tqdm
!pip install parameter-sherpa

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import sherpa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

torch.__version__

'1.3.1'

In [0]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [0]:
path = "drive/My Drive/AML/"
#path = ""

df = pd.read_csv(path + "pmsm_temperature_data.csv")

In [0]:
df_sep = [df[df
             .profile_id==profile]
          .drop(['profile_id','torque','stator_yoke',
                 'stator_tooth','stator_winding'],axis=1)
          .reset_index(drop=True) for profile in df.profile_id.unique()]

In [0]:
def outputSize(in_size, kernel_size, stride, padding):
  output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
  return(output)

class CNN_Net_2(nn.Module):
    def __init__(self, batch, in_c, out, 
                 kernel1, kernel2, kernel3, kernel4, 
                 #padding1, padding2, padding3, padding4, 
                 stride1, stride2, stride3, stride4):
        super(CNN_Net_2, self).__init__()
        self.batch_size = 1
        self.in_c = in_c
        l0 = 60
        l1 = outputSize(l0,kernel1,stride1,0)
        l2 = outputSize(l1,kernel2,stride2,0)
        l3 = outputSize(l2,kernel3,stride3,0)
        l4 = outputSize(l3,kernel4,stride4,0)
        #print(l4)

        self.conv1 = nn.Conv1d(in_channels= in_c, 
                               out_channels= 16, 
                               kernel_size= kernel1,
                               stride= stride1,
                               padding=0)    
        self.pool1 = nn.MaxPool1d(kernel2,stride=stride2,padding=0) 
        self.conv2 = nn.Conv1d(16,8,kernel3,stride=stride3,padding=0)
        self.pool2 = nn.MaxPool1d(kernel4,stride=stride4,padding=0)
        self.fc1 = nn.Linear(8*l4, 100)
        self.fc2 = nn.Linear(100, out)

    def forward(self, x):
        batch_size, _, _ = x.shape
        x =  self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(batch_size, self.num_flat_features(x))
        x = F.relu(self.fc1(x)) 
        return  self.fc2(x)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:       # Get the products
            num_features *= s
        return num_features

In [0]:
def sliding_window(dataset, target_var, inp, out, shuffle=True):
    while True:
        for X in dataset:
            target = X[[target_var]]
            num_features = len(X.columns)
            
            indice = list(range(inp + out, X.shape[0]))
            
            if shuffle:
                np.random.shuffle(indice)
            for i in indice:
                features = X.iloc[i-inp-out:i-out,].values.reshape(inp,num_features)
                pred = target.iloc[i:i+out]

                yield np.array(features), np.array(pred)

In [0]:
FEATURES = [0, 1, 2, 3, 4, 5, 6]
TARGET = [7]

In [0]:
def dataloader(data, length, shuffle=True, out=1):
    while True:
    # genera una lista di (i_serie, i_obs)
        tuples = [[(df_i, i) for i, x in enumerate(data[df_i]) if i >= length + out]
                  for df_i, _ in enumerate(data)]
        tuples = sum(tuples, [])  # flattenizza
        # shuffle
        if shuffle:
            np.random.shuffle(tuples)

        # yielda le osservazioni
        for df_i, i in tuples:
            X_lagged = data[df_i][(i - length - out):(i - out), FEATURES + TARGET]
            y = data[df_i][(i-out):(i), TARGET]
            yield X_lagged, y

In [0]:
train_min = df[~df.profile_id.isin([4, 8, 16, 24, 32, 40, 48, 51, 35, 42])].drop(['profile_id','torque',
                                                                                 'stator_yoke','stator_tooth',
                                                                                  'stator_winding'], 
                                                                                 axis=1).min()
train_max = df[~df.profile_id.isin([4, 8, 16, 24, 32, 40, 48, 51, 35, 42])].drop(['profile_id','torque',
                                                                                 'stator_yoke','stator_tooth',
                                                                                  'stator_winding'], 
                                                                                 axis=1).max()

In [0]:
df_train = [(df_sep[i] - train_min)/(train_max - train_min)\
            for i in list(range(0,52)) if i not in [4, 8, 16, 24, 32, 
                                                    40, 48, 51, #val
                                                    35, 42]] #test

df_val = [(df_sep[i] - train_min)/(train_max - train_min)\
          for i in [4, 8, 16, 24, 32, 40, 48, 51]]

df_test = [(df_sep[i] - train_min)/(train_max - train_min) for i in [35, 42]]

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [0]:
parameters = [sherpa.Continuous('lr',[0.005,0.05]),
              sherpa.Discrete('kernel1',[1,5]),
              sherpa.Discrete('kernel2',[1,5]),
              sherpa.Discrete('kernel3',[1,5]),
              sherpa.Discrete('kernel4',[1,5]),
              sherpa.Discrete('stride1',[1,3]),
              sherpa.Discrete('stride2',[1,3]),
              sherpa.Discrete('stride3',[1,3]),
              sherpa.Discrete('stride4',[1,3]),
              #sherpa.Discrete('padding1',[0,1]),
              #sherpa.Discrete('padding2',[0,1]),
              #sherpa.Discrete('padding3',[0,1]),
              #sherpa.Discrete('padding4',[0,1]),
              sherpa.Choice('batch_size',[512, 1024, 2048])]

alg = sherpa.algorithms.bayesian_optimization.GPyOpt(max_concurrent=1,
                                         model_type='GP',
                                         acquisition_type='EI',
                                         max_num_trials=100)

study = sherpa.Study(parameters=parameters,
                     algorithm=alg,
                     lower_is_better=True,
                     disable_dashboard=True)

In [26]:
look_back  = 60
best_score = 99999
feature    = 8

for trial in study:

    lstm = CNN_Net_2(batch=int(trial.parameters["batch_size"]),
                     in_c=feature,
                     out = 1,
                     kernel1=int(trial.parameters["kernel1"]),
                     kernel2=int(trial.parameters["kernel2"]),
                     kernel3=int(trial.parameters["kernel3"]),
                     kernel4=int(trial.parameters["kernel4"]),
                     stride1=int(trial.parameters["stride1"]),
                     stride2=int(trial.parameters["stride2"]),
                     stride3=int(trial.parameters["stride3"]),
                     stride4=int(trial.parameters["stride4"]),
                     #padding1=int(trial.parameters["padding1"]),
                     #padding2=int(trial.parameters["padding2"]),
                     #padding3=int(trial.parameters["padding3"]),
                     #padding4=int(trial.parameters["padding4"])
                     ).to(device)
                     
    losses = []
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(lstm.parameters(), lr=trial.parameters["lr"])

    batch_size = int(trial.parameters["batch_size"])

    df_length = np.sum([x.shape[0] for x in df_train])

    #gen = sliding_window(df_train, "pm", look_back, 1)
    gen = dataloader([df.values for df in df_train], look_back)
    progress_bar = tqdm(range(0, df_length, batch_size))

    lstm.train()
    for b in progress_bar:
        X_train = []
        y_train = []
        for i in range(batch_size):
            try:
                X,y = next(gen)
                X_train.append(X)
                y_train.append(y)
            except StopIteration:
                #in case the datagen ends, recreate a new a one and continue
                #it should not happen though since the the datagen
                #should be infinite (theorically), it's more of a precaution
                #gen = sliding_window(df_train, "pm", look_back, 1)
                gen = dataloader([df.values for df in df_train], look_back)

                X,y = next(gen)
                X_train.append(X)
                y_train.append(y)
        
        inpt = np.array(X_train).reshape(-1, 8, look_back)
        target = np.array(y_train)    
        x_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
        y_batch = torch.tensor(target,dtype=torch.float32).to(device)
        try:
            output = lstm(x_batch) 
            loss = criterion(output.view(-1), y_batch.view(-1))  

            loss.backward()
            optimizer.step()        
            optimizer.zero_grad()

            losses.append(loss.item())
        except:
            print("something strange happened")
            print(inpt.shape)
            print(x_batch.size())
            break
        if len(losses)>10000:
            progress_bar.set_description("Epoch:"+str(int(b/batch_size)+1)+" Loss:"+str(round(np.mean(losses[-1000,]),7)))
        else:
            progress_bar.set_description("Epoch:"+str(int(b/batch_size)+1)+" Loss:"+str(round(np.mean(losses),7)))

    #test_gen = sliding_window(df_val, "pm", look_back, 1)
    test_gen = dataloader([df.values for df in df_val], look_back)
    batch_size = 510

    lstm.eval()
    y_test = []
    y_pred_all = []

    
    tot_len = np.sum([x.shape[0] for x in df_val])
    for x in range(0, tot_len, batch_size):
        X_test = []
        for i in range(batch_size):
            try:
                X,y = next(test_gen)
                X_test.append(X)
                y_test.append(y)
            except:
                print("You somehow created an exception hahaha!")
                break

        inpt = np.array(X_test).reshape(-1, 8, look_back)
        x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
        #print(x_test_batch.shape)
        y_pred = lstm(x_test_batch)

        y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())

    y_test = np.array(y_test).reshape(-1)
    score = np.mean((y_test - y_pred_all)**2) #MSE
    if score < best_score:
        best_score = score
        torch.save(lstm, path + "models/best_model_cnn")
        print("New Challanger with MSE on val:", score, "and confs:", trial.parameters)
    
    #Sherpa PART
    study.add_observation(trial, iteration=1, objective=score)
    study.finalize(trial)
    study.results.to_csv(path + "results/AutoML_CNN.csv")

  0%|          | 0/1506 [00:00<?, ?it/s]

New Challanger with MSE on val: 0.025155118690731428 and confs: {'lr': 0.006999504608353264, 'kernel1': 1, 'kernel2': 2, 'kernel3': 1, 'kernel4': 3, 'stride1': 2, 'stride2': 2, 'stride3': 2, 'stride4': 1, 'batch_size': 1024}


  0%|          | 0/753 [00:00<?, ?it/s]

New Challanger with MSE on val: 0.010386161364523014 and confs: {'lr': 0.029963974431836453, 'kernel1': 2, 'kernel2': 3, 'kernel3': 4, 'kernel4': 3, 'stride1': 2, 'stride2': 1, 'stride3': 1, 'stride4': 1, 'batch_size': 2048}


  0%|          | 0/377 [00:00<?, ?it/s]

...

New Challanger with MSE on val: 0.0004233472185203098 and confs: {'lr': 0.005, 'kernel1': 3.0, 'kernel2': 3.0, 'kernel3': 1.0, 'kernel4': 1.0, 'stride1': 1.0, 'stride2': 1.0, 'stride3': 2.0, 'stride4': 1.0, 'batch_size': 1024.0}


Epoch:753 Loss:0.0021943: 100%|██████████| 753/753 [00:25<00:00, 29.52it/s]
INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters


New Challanger with MSE on val: 0.0003349143275392701 and confs: {'lr': 0.005, 'kernel1': 3.0, 'kernel2': 3.0, 'kernel3': 1.0, 'kernel4': 1.0, 'stride1': 1.0, 'stride2': 1.0, 'stride3': 2.0, 'stride4': 1.0, 'batch_size': 1024.0}


Epoch:753 Loss:0.0022943: 100%|██████████| 753/753 [00:25<00:00, 29.38it/s]
INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters
Epoch:753 Loss:0.0021103: 100%|██████████| 753/753 [00:24<00:00, 12.28it/s]
INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters
Epoch:753 Loss:0.0035246: 100%|██████████| 753/753 [00:25<00:00, 29.84it/s]
INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters
Epoch:753 Loss:0.0031363: 100%|██████████| 753/753 [00:24<00:00, 30.16it/s]
INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters
Epoch:753 Loss:0.0022333: 100%|██████████| 753/753 [00:25<00:00, 29.97it/s]
INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters
Epoch:753 Loss:0.0034347: 100%|██████████| 753/753 [00:24<00

Buffered data was truncated after reaching the output size limit.

In [0]:
best_model = torch.load(path + "models/best_model_cnn")
results = pd.read_csv(path + "results/AutoML_CNN.csv")
results = results[results["Status"]=="COMPLETED"]

In [0]:
plt.figure(figsize=(15,5))
best, = plt.plot(np.minimum.accumulate(np.array(results.Objective)),'*-')
#actual, = plt.plot(np.array(results.Objective),'*-')

#plt.legend([actual, best],
#           ['Calculated','Best Seen']) 

plt.xlabel('Iterations')
plt.ylabel('Best Seen MSE')
plt.show()

In [0]:
test_gen = dataloader([df_test[0].values], look_back, shuffle=False)

batch_size = 510

y_test = []
y_pred_all = []
for x in range(0, df_test[0].shape[0], batch_size):
    X_test = []
    for i in range(batch_size):
        try:
            X,y = next(test_gen)
            X_test.append(X)
            y_test.append(y)
        except:
            break

    inpt = np.array(X_test).reshape(-1, 8, look_back)
    x_test_batch = torch.tensor(inpt,dtype=torch.float32).to(device)  
    y_pred = best_model(x_test_batch)

    y_pred_all = np.append(y_pred_all,y_pred.cpu().detach().numpy())

In [0]:
y_test = np.array(y_test).reshape(-1)
plt.figure(figsize=(10,10))
plt.plot(y_test[-10:])
plt.plot(y_pred_all[-10:])

plt.show()

In [0]:
np.mean(np.abs((y_test - y_pred_all)/np.abs(y_test)))